<a href="https://colab.research.google.com/github/cassiegraf/deep-learning-challenge/blob/main/AlphabetSoupCharity_Optimization3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Optimization attempt #3 (after running original data with 72% accuracy).**


*   In addition to 'EIN' and 'NAME' columns, also dropped 'ASK_AMT' because there was over 8,000 unique values.
*   'APPLICATION_TYPE' and 'CLASSIFICATION' cutoffs remain the same as in the orginal model.
* Adjusted epochs to 75 since the other models hit their accuracy rate far before 100 epochs
* Attempt #3 returned the same resutls as the original mode, attempt #1, attempt #2, with an accuracy of 72%


In [3]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


**Target Variables:**

*   IS_SUCCESSFUL


**Feature Variables:**   

*   APPLICATION_TYPE
*   AFFILIATION
*   CLASSIFICATION
*   USE_CAUSE
*   ORGANIZATION
*   STATUS
*   INCOME_AMT
*   SPECIAL_CONSIDERATIONS
*   ASK_AMT (not including in optimization #3)

In [4]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
cols_keep = ['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION',
       'USE_CASE', 'ORGANIZATION', 'STATUS', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS', 'IS_SUCCESSFUL']
funding_df = application_df[cols_keep]
funding_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,1


In [5]:
# Look at APPLICATION_TYPE value counts for binning
app_count = funding_df["APPLICATION_TYPE"].value_counts()
app_count

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Choose a cutoff value (less than 527) and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(app_count[app_count < 527].index)

# Replace in dataframe
for app in application_types_to_replace:
    funding_df['APPLICATION_TYPE'] = funding_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
funding_df['APPLICATION_TYPE'].value_counts()

<ipython-input-6-81140e655f0b>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  funding_df['APPLICATION_TYPE'] = funding_df['APPLICATION_TYPE'].replace(app,"Other")


T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [7]:
class_count = funding_df["CLASSIFICATION"].value_counts()

In [8]:
# Choose a cutoff value (greater than 1882) and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(class_count[class_count < 1882].index)

# Replace in dataframe
for app in classifications_to_replace:
    funding_df['CLASSIFICATION'] = funding_df['CLASSIFICATION'].replace(app,"Other")

# Check to make sure binning was successful
funding_df['CLASSIFICATION'].value_counts()

<ipython-input-8-1f5792729f6f>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  funding_df['CLASSIFICATION'] = funding_df['CLASSIFICATION'].replace(app,"Other")


C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [11]:
# Convert categorical data to numeric with `pd.get_dummies`
dummy = pd.get_dummies(funding_df)

In [13]:
funding_df_final = dummy

In [14]:
# Split our preprocessed data into our features and target arrays
y = funding_df_final['IS_SUCCESSFUL']
X = funding_df_final.drop(columns='IS_SUCCESSFUL')

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [15]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [16]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 80
hidden_nodes_layer2 = 30


nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim = input_features, activation = "relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation = "relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                3440      
                                                                 
 dense_1 (Dense)             (None, 30)                2430      
                                                                 
 dense_2 (Dense)             (None, 1)                 31        
                                                                 
Total params: 5,901
Trainable params: 5,901
Non-trainable params: 0
_________________________________________________________________


In [17]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [18]:
# Train the model
fit_model=nn.fit(X_train_scaled, y_train, epochs=75)

Epoch 1/75
804/804 [==============================] - 4s 3ms/step - loss: 0.5680 - accuracy: 0.7207
Epoch 2/75
804/804 [==============================] - 2s 3ms/step - loss: 0.5525 - accuracy: 0.7320
Epoch 3/75
804/804 [==============================] - 2s 3ms/step - loss: 0.5497 - accuracy: 0.7313
Epoch 4/75
804/804 [==============================] - 1s 2ms/step - loss: 0.5482 - accuracy: 0.7322
Epoch 5/75
804/804 [==============================] - 2s 2ms/step - loss: 0.5474 - accuracy: 0.7332
Epoch 6/75
804/804 [==============================] - 6s 7ms/step - loss: 0.5465 - accuracy: 0.7332
Epoch 7/75
804/804 [==============================] - 3s 4ms/step - loss: 0.5456 - accuracy: 0.7345
Epoch 8/75
804/804 [==============================] - 3s 4ms/step - loss: 0.5454 - accuracy: 0.7353
Epoch 9/75
804/804 [==============================] - 3s 3ms/step - loss: 0.5451 - accuracy: 0.7352
Epoch 10/75
804/804 [==============================] - 3s 4ms/step - loss: 0.5442 - accuracy: 0.7343

In [19]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5649 - accuracy: 0.7259 - 478ms/epoch - 2ms/step
Loss: 0.5649238228797913, Accuracy: 0.7259474992752075


In [20]:
# Export our model to HDF5 file
nn.save(r"C:\Users\cassi\OneDrive\deep-learning-challenge\AlphabetSoupCharity_Optimization2.h5")